In [1]:
import geopandas
import scipy.io
import pandas as pd
import numpy as np
from EchoPro import EchoPro
epro_2019 = EchoPro(init_file_path='./config_files/initialization_config.yml',
                    survey_year_file_path='./config_files/survey_year_2019_config.yml',
                    source=3,
                    bio_data_type=1,
                    age_data_status=1, 
                    exclude_age1=True)

A check of the initialization file needs to be done!
A check of the survey year file needs to be done!
Loading biological data ...


/Users/brandonreyes/UW_work/EchoPro_work/EchoPro/EchoPro/echo_pro.py:274: UserWarning: We are currently using wgt_total from Matlab for CV, change this!
  warnings.warn("We are currently using wgt_total from Matlab for CV, change this!")


In [2]:
krig_mesh = epro_2019.get_kriging_mesh()


In [3]:
# get geopandas representation of final_biomass_table
# allows us to plot transect data
df = epro_2019.final_biomass_table
gdf = geopandas.GeoDataFrame(df, 
                             geometry=geopandas.points_from_xy(df.Longitude, df.Latitude))

# reseting the index so that we can select data based on column name
gdf = gdf.reset_index()

## Plots transect points and the full mesh 

In [4]:
# # Plot transect points 
# fmap = krig_mesh.plot_points(gdf, cmap_column='Transect', color='hex')

# # Plot full mesh points 
# fmap = krig_mesh.plot_points(krig_mesh.mesh_gdf, fmap, color='red')

# # display the folium map
# fmap

In [5]:
# transform the data using the Python version of EchoPro
# transformed_transect_gdf = krig_mesh.apply_longitude_transformation(gdf)
# transformed_mesh_gdf = krig_mesh.apply_longitude_transformation(krig_mesh.mesh_gdf, 
#                                                                 gdf_lon_name="Longitude of centroid", 
#                                                                 gdf_lat_name="Latitude of centroid")

In [6]:
# get kriging input data from Matlab version of EchoPro
krig_input_mat = scipy.io.loadmat('../2019_consolidated_files/kriging_input_points.mat')

x_data = krig_input_mat['xn'].flatten()
y_data = krig_input_mat['yn'].flatten()
field_data = krig_input_mat['var'].flatten()

x_mesh = krig_input_mat['xp'].flatten()
y_mesh = krig_input_mat['yp'].flatten()

In [7]:
%%time
# compute the distance between the mesh points and transect points
x_diff = np.subtract.outer(x_mesh, x_data)
y_diff = np.subtract.outer(y_mesh, y_data)

dis = np.sqrt(x_diff*x_diff + y_diff*y_diff)

CPU times: user 1.03 s, sys: 2.26 s, total: 3.29 s
Wall time: 4.62 s


In [8]:
%%time
kmax = 10
k_min = 3

# sort dis up to the kmax smallest elements in each row
dis_sort_ind = np.argpartition(dis, kmax, axis=1)

# select only the kmax smallest elements in each row 
dis_kmax_ind = dis_sort_ind[:, :kmax]

# sort the columns so they are in ascending order
# dis_kmax = np.sort(dis_kmax, axis=1)  # TODO: is this necessary?

CPU times: user 728 ms, sys: 258 ms, total: 986 ms
Wall time: 995 ms


In [9]:
%%time
R = 0.0226287
ratio = 0.001
nugget = 0.0
sill = 0.95279
ls = 0.0075429
exp_pow = 1.5
ls_hole_eff = 0.0

from EchoPro.semivariogram import SemiVariogram as SV

ep = []

# does Ordinary Kriging, follow Journel and Huijbregts, p. 307
for row in range(dis_kmax_ind.shape[0]):
    
    sel_ind = dis_kmax_ind[row, :]
    R_ind = np.argwhere(dis[row, sel_ind] <= R).flatten()
    
    if len(R_ind) < k_min:
        R_ind = np.argsort(dis[row, sel_ind]).flatten()[:k_min]
    
    # TODO: put in the M2_unity statement

    
    # TODO: replace with automatic call to correct model
    M20 = SV.generalized_exp_bessel(dis[row, sel_ind[R_ind]], sill, ls, exp_pow, ls_hole_eff, nugget)
    
    M2 = np.concatenate([M20, np.array([1.0])]) # for Ordinary Kriging
    
#     print(f"M2 = {M2}")
    
    x1 = x_data[sel_ind[R_ind]]
    y1 = y_data[sel_ind[R_ind]]
    
    # compute the distance between the points
    x1_diff = np.subtract.outer(x1, x1)
    y1_diff = np.subtract.outer(y1, y1)
    dis1 = np.sqrt(x1_diff*x1_diff + y1_diff*y1_diff)
    
    # TODO: replace with automatic call to correct model
    K0 = SV.generalized_exp_bessel(dis1, sill, ls, exp_pow, ls_hole_eff, nugget)
    
    # Add column and row of ones for Ordinary Kriging
    K = np.concatenate([K0, np.ones((len(x1), 1))], axis=1)
    K = np.concatenate([K, np.ones((1, len(x1) + 1))], axis=0)
    
    # do an inplace fill of diagonal
    np.fill_diagonal(K, 0.0)
    
    # compute SVD
    u, s, vh = np.linalg.svd(K, full_matrices=True)

    kindx = np.argwhere(np.abs(s/s[0]) > ratio).flatten()
    
    s_inv = 1.0/s[kindx]

    k_inv = np.matmul(vh.T[:, kindx], np.diag(s_inv))
    k_inv = np.matmul(k_inv, u[:, kindx].T)
    
    lamb = np.dot(k_inv, M2)

#     Vp(i)=sum_nan(lambda(1:nk).*var1)*M2_unity;

    ep.append(np.nansum(lamb*M2))
    
# print(ep)
    
#     import sys 
#     sys.exit()
    



CPU times: user 2.5 s, sys: 21.5 ms, total: 2.53 s
Wall time: 2.53 s


In [10]:
xn = np.array([1, 2, 3])
xp = np.array([0.5, 1.5, 2.5, 3.5])

np.subtract.outer(xp, xn)

array([[-0.5, -1.5, -2.5],
       [ 0.5, -0.5, -1.5],
       [ 1.5,  0.5, -0.5],
       [ 2.5,  1.5,  0.5]])

In [11]:
x_data

array([-0.14360447, -0.1414857 , -0.13938711, ...,  0.00529606,
        0.00737726,  0.00917433])